In [1]:
import os

import torch

from datasets import load_dataset

from transformers import AutoModelForSequenceClassification, AutoTokenizer

from peft import get_peft_model, TaskType

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.environ["WANDB_PROJECT"] = "Llama-3.2-1B-Instruct-Reward"
os.environ["WANDB_ENTITY"] = "RADFAN"

In [2]:
# =============================================================================
# Configs
# =============================================================================

# Model config
# =============================================================================

model_config = ModelConfig(
    model_name_or_path   = "meta-llama/Llama-3.2-1B-Instruct",
    # LoRA
    # -------------------------------------------------------------------------
    use_peft             = True,
    lora_task_type       = TaskType.SEQ_CLS,
    lora_r               = 8,
    lora_alpha           = 8,
    lora_dropout         = 0.05,
    lora_target_modules  = None,
    lora_modules_to_save = None,
    # Quantization
    # -------------------------------------------------------------------------
    load_in_8bit         = False,
    load_in_4bit         = False,
    torch_dtype          = "bfloat16"
)

# Reward trainer config
# =============================================================================

training_args = RewardConfig(
    # Reward trainer params
    # -------------------------------------------------------------------------
    max_length                  = None,
    dataset_num_proc            = 16,
    center_rewards_coefficient  = None,
    # Common
    # -------------------------------------------------------------------------
    output_dir                  = f"{model_config.model_name_or_path}-reward",
    run_name                    = f"{model_config.model_name_or_path}-{model_config.lora_r}",
    per_device_train_batch_size = 4,
    num_train_epochs            = 2,
    gradient_checkpointing      = False,
    gradient_accumulation_steps = 4,
    learning_rate               = 1e-4,

    # Frequency of logs
    # -------------------------------------------------------------------------
    logging_steps               = 25,

    # Evaluation
    # -------------------------------------------------------------------------
    eval_strategy               = "steps",
    eval_steps                  = 125,

    # Push to hub after training
    # -------------------------------------------------------------------------
    push_to_hub                 = True,
    hub_model_id                = "RLHF-And-Friends/Llama-3.2-1B-Instruct-Reward"
)

# Script config
# =============================================================================

script_args = ScriptArguments(
    dataset_name        = "trl-lib/ultrafeedback_binarized",
    dataset_train_split = "train",
    dataset_test_split  = "test",
)

In [3]:
# =============================================================================
# Model & Tokenizer
# =============================================================================

# Tokenizer
# -----------------------------------------------------------------------------

tokenizer = AutoTokenizer.from_pretrained(
    model_config.model_name_or_path, use_fast=True
)
tokenizer.pad_token = tokenizer.eos_token

# Model
# -----------------------------------------------------------------------------

quantization_config = get_quantization_config(model_config)
lora_config = get_peft_config(model_config)
device_map = (
    get_kbit_device_map() if quantization_config is not None else "auto"
)
use_cache = False if training_args.gradient_checkpointing else True
torch_dtype = (
    getattr(
        torch, model_config.torch_dtype
    ) if model_config.torch_dtype is not None 
    else None
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_config.model_name_or_path, 
    num_labels = 1,
    quantization_config = quantization_config,
    device_map = device_map,
    use_cache = use_cache,
    torch_dtype = torch_dtype
)

model = get_peft_model(model, lora_config)

if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
     

In [5]:
# =============================================================================
# Load dataset
# =============================================================================

dataset = load_dataset(script_args.dataset_name)

train_dataset = dataset[script_args.dataset_train_split]
eval_dataset = (
    dataset[script_args.dataset_test_split] 
    if training_args.eval_strategy != "no" 
    else None
)

In [6]:
# =============================================================================
# Training
# =============================================================================

trainer = RewardTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: evgurovv (RADFAN). Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
125,0.693300,0.688198,0.756098
250,0.695900,0.701749,0.300000
375,0.694200,0.686373,0.634146
500,0.692600,0.684271,0.634146
625,0.693400,0.698431,0.333333
750,0.692700,0.690134,0.684211
875,0.691600,0.694172,0.205128
1000,0.693200,0.685709,0.666667
1125,0.690200,0.687479,0.820513
1250,0.693400,0.689249,0.682927


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|begin_of_text|><|begin_of_text|><|start_he… │ <|begin_of_text|><|begin_of_text|><|start_h… │ [0.4805, 0.5195] │
│                                               │                                              │                  │
│ Cutting Knowledge Date: December 2023         │ Cutting Knowledge Date: December 2023        │                  │
│ Today Date: 19 Dec 2024                       │ Today Date: 19 Dec 2024                      │                  │
│                                               │                                              │                  │
│ <|start_header_id|>user<|end_header_id|>      │ <|start_header_id|>user<|end_header_id|>     │                  │
│                                               │                                              │                  │
│ Definition: In this task, you are given a     │ Definition: In this task, you are given a    │                  │
│ summary for US Congressional and California   │ summary for US Congressional and California  │                  │
│ state bill, your task is to generate a Title  │ state bill, your task is to generate a Title │                  │
│ for this bill. The preferred titles are under │ for this bill. The preferred titles are      │                  │
│ forty words and mention the purpose of the    │ under forty words and mention the purpose of │                  │
│ bill.                                         │ the bill.                                    │                  │
│ Input: Intelligent Technologies Initiative    │ Input: Intelligent Technologies Initiative   │                  │
│ Act of 2015  This bill directs the Department │ Act of 2015  This bill directs the           │                  │
│ of Transportation (DOT) to establish an       │ Department of Transportation (DOT) to        │                  │
│ Intelligent Technology Initiative to provide  │ establish an Intelligent Technology          │                  │
│ grants to up to six state and local           │ Initiative to provide grants to up to six    │                  │
│ governments to establish deployment sites for │ state and local governments to establish     │                  │
│ large scale installation and operation of     │ deployment sites for large scale             │                  │
│ intelligent transportation systems (ITS) to   │ installation and operation of intelligent    │                  │
│ improve safety, efficiency, system            │ transportation systems (ITS) to improve      │                  │
│ performance, and return on investment. DOT    │ safety, efficiency, system performance, and  │                  │
│ shall develop criteria for selecting grant    │ return on investment. DOT shall develop      │                  │
│ recipients, including how the deployment of   │ criteria for selecting grant recipients,     │                  │
│ technology will enable the recipient to:      │ including how the deployment of technology   │                  │
│ reduce costs and improve return on            │ will enable the recipient to:   reduce costs │                  │
│ investments;   deliver environmental benefits │ and improve return on investments;   deliver │                  │
│ and reduce energy consumption by alleviating  │ environmental benefits and reduce energy     │                  │
│ congestion and streamlining traffic flow;     │ consumption by alleviating congestion and    │                  │
│ measure and improve the operational           │ streamlining traffic flow;   measure and     │                  │
│ performance of its transportation network;    │ improv

/home/gurovev/dev/rlhf-and-friends/FedPPO/.fed-ppo-venv/lib/python3.10/site-packages/trl/trainer/utils.py:785: UserWarning: There are 237 out of 565 instances where the predictions for both options are equal. These instances are ignored in the accuracy computation.
  warnings.warn(


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|begin_of_text|><|begin_of_text|><|start_he… │ <|begin_of_text|><|begin_of_text|><|start_h… │ [0.5, 0.5]       │
│                                               │                                              │                  │
│ Cutting Knowledge Date: December 2023         │ Cutting Knowledge Date: December 2023        │                  │
│ Today Date: 19 Dec 2024                       │ Today Date: 19 Dec 2024                      │                  │
│                                               │                                              │                  │
│ <|start_header_id|>user<|end_header_id|>      │ <|start_header_id|>user<|end_header_id|>     │                  │
│                                               │                                              │                  │
│ Definition: In this task, you are given a     │ Definition: In this task, you are given a    │                  │
│ summary for US Congressional and California   │ summary for US Congressional and California  │                  │
│ state bill, your task is to generate a Title  │ state bill, your task is to generate a Title │                  │
│ for this bill. The preferred titles are under │ for this bill. The preferred titles are      │                  │
│ forty words and mention the purpose of the    │ under forty words and mention the purpose of │                  │
│ bill.                                         │ the bill.                                    │                  │
│ Input: Intelligent Technologies Initiative    │ Input: Intelligent Technologies Initiative   │                  │
│ Act of 2015  This bill directs the Department │ Act of 2015  This bill directs the           │                  │
│ of Transportation (DOT) to establish an       │ Department of Transportation (DOT) to        │                  │
│ Intelligent Technology Initiative to provide  │ establish an Intelligent Technology          │                  │
│ grants to up to six state and local           │ Initiative to provide grants to up to six    │                  │
│ governments to establish deployment sites for │ state and local governments to establish     │                  │
│ large scale installation and operation of     │ deployment sites for large scale             │                  │
│ intelligent transportation systems (ITS) to   │ installation and operation of intelligent    │                  │
│ improve safety, efficiency, system            │ transportation systems (ITS) to improve      │                  │
│ performance, and return on investment. DOT    │ safety, efficiency, system performance, and  │                  │
│ shall develop criteria for selecting grant    │ return on investment. DOT shall develop      │                  │
│ recipients, including how the deployment of   │ criteria for selecting grant recipients,     │                  │
│ technology will enable the recipient to:      │ including how the deployment of technology   │                  │
│ reduce costs and improve return on            │ will enable the recipient to:   reduce costs │                  │
│ investments;   deliver environmental benefits │ and improve return on investments;   deliver │                  │
│ and reduce energy consumption by alleviating  │ environmental benefits and reduce energy     │                  │
│ congestion and streamlining traffic flow;     │ consumption by alleviating congestion and    │                  │
│ measure and improve the operational           │ streamlining traffic flow;   measure and     │                  │
│ performance of its transportation network;    │ improv

/home/gurovev/dev/rlhf-and-friends/FedPPO/.fed-ppo-venv/lib/python3.10/site-packages/trl/trainer/utils.py:785: UserWarning: There are 325 out of 565 instances where the predictions for both options are equal. These instances are ignored in the accuracy computation.
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|begin_of_text|><|begin_of_text|><|start_he… │ <|begin_of_text|><|begin_of_text|><|start_h… │ [0.5078, 0.4922] │
│                                               │                                              │                  │
│ Cutting Knowledge Date: December 2023         │ Cutting Knowledge Date: December 2023        │                  │
│ Today Date: 19 Dec 2024                       │ Today Date: 19 Dec 2024                      │                  │
│                                               │                                              │                  │
│ <|start_header_id|>user<|end_header_id|>      │ <|start_header_id|>user<|end_header_id|>     │                  │
│                                               │                                              │                  │
│ Definition: In this task, you are given a     │ Definition: In this task, you are given a    │                  │
│ summary for US Congressional and California   │ summary for US Congressional and California  │                  │
│ state bill, your task is to generate a Title  │ state bill, your task is to generate a Title │                  │
│ for this bill. The preferred titles are under │ for this bill. The preferred titles are      │                  │
│ forty words and mention the purpose of the    │ under forty words and mention the purpose of │                  │
│ bill.                                         │ the bill.                                    │                  │
│ Input: Intelligent Technologies Initiative    │ Input: Intelligent Technologies Initiative   │                  │
│ Act of 2015  This bill directs the Department │ Act of 2015  This bill directs the           │                  │
│ of Transportation (DOT) to establish an       │ Department of Transportation (DOT) to        │                  │
│ Intelligent Technology Initiative to provide  │ establish an Intelligent Technology          │                  │
│ grants to up to six state and local           │ Initiative to provide grants to up to six    │                  │
│ governments to establish deployment sites for │ state and local governments to establish     │                  │
│ large scale installation and operation of     │ deployment sites for large scale             │                  │
│ intelligent transportation systems (ITS) to   │ installation and operation of intelligent    │                  │
│ improve safety, efficiency, system            │ transportation systems (ITS) to improve      │                  │
│ performance, and return on investment. DOT    │ safety, efficiency, system performance, and  │                  │
│ shall develop criteria for selecting grant    │ return on investment. DOT shall develop      │                  │
│ recipients, including how the deployment of   │ criteria for selecting grant recipients,     │                  │
│ technology will enable the recipient to:      │ including how the deployment of technology   │                  │
│ reduce costs and improve return on            │ will enable the recipient to:   reduce costs │                  │
│ investments;   deliver environmental benefits │ and improve return on investments;   deliver │                  │
│ and reduce energy consumption by alleviating  │ environmental benefits and reduce energy     │                  │
│ congestion and streamlining traffic flow;     │ consumption by alleviating congestion and    │                  │
│ measure and improve the operational           │ streamlining traffic flow;   measure and     │                  │
│ performance of its transportation network;    │ improv

/home/gurovev/dev/rlhf-and-friends/FedPPO/.fed-ppo-venv/lib/python3.10/site-packages/trl/trainer/utils.py:785: UserWarning: There are 309 out of 565 instances where the predictions for both options are equal. These instances are ignored in the accuracy computation.
  warnings.warn(


TrainOutput(global_step=4386, training_loss=0.6932556180175559, metrics={'train_runtime': 8895.0899, 'train_samples_per_second': 7.892, 'train_steps_per_second': 0.493, 'total_flos': 0.0, 'train_loss': 0.6932556180175559, 'epoch': 1.9996581196581196})

In [7]:
# =============================================================================
# Save model and push to Hub
# =============================================================================

trainer.save_model(training_args.output_dir)
if training_args.push_to_hub:
    trainer.push_to_hub(dataset_name=script_args.dataset_name)

In [8]:
# =============================================================================
# Evaluate Model
# =============================================================================

if training_args.eval_strategy != "no":
    metrics = trainer.evaluate()
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|begin_of_text|><|begin_of_text|><|start_he… │ <|begin_of_text|><|begin_of_text|><|start_h… │ [0.5195, 0.4805] │
│                                               │                                              │                  │
│ Cutting Knowledge Date: December 2023         │ Cutting Knowledge Date: December 2023        │                  │
│ Today Date: 19 Dec 2024                       │ Today Date: 19 Dec 2024                      │                  │
│                                               │                                              │                  │
│ <|start_header_id|>user<|end_header_id|>      │ <|start_header_id|>user<|end_header_id|>     │                  │
│                                               │                                              │                  │
│ Definition: In this task, you are given a     │ Definition: In this task, you are given a    │                  │
│ summary for US Congressional and California   │ summary for US Congressional and California  │                  │
│ state bill, your task is to generate a Title  │ state bill, your task is to generate a Title │                  │
│ for this bill. The preferred titles are under │ for this bill. The preferred titles are      │                  │
│ forty words and mention the purpose of the    │ under forty words and mention the purpose of │                  │
│ bill.                                         │ the bill.                                    │                  │
│ Input: Intelligent Technologies Initiative    │ Input: Intelligent Technologies Initiative   │                  │
│ Act of 2015  This bill directs the Department │ Act of 2015  This bill directs the           │                  │
│ of Transportation (DOT) to establish an       │ Department of Transportation (DOT) to        │                  │
│ Intelligent Technology Initiative to provide  │ establish an Intelligent Technology          │                  │
│ grants to up to six state and local           │ Initiative to provide grants to up to six    │                  │
│ governments to establish deployment sites for │ state and local governments to establish     │                  │
│ large scale installation and operation of     │ deployment sites for large scale             │                  │
│ intelligent transportation systems (ITS) to   │ installation and operation of intelligent    │                  │
│ improve safety, efficiency, system            │ transportation systems (ITS) to improve      │                  │
│ performance, and return on investment. DOT    │ safety, efficiency, system performance, and  │                  │
│ shall develop criteria for selecting grant    │ return on investment. DOT shall develop      │                  │
│ recipients, including how the deployment of   │ criteria for selecting grant recipients,     │                  │
│ technology will enable the recipient to:      │ including how the deployment of technology   │                  │
│ reduce costs and improve return on            │ will enable the recipient to:   reduce costs │                  │
│ investments;   deliver environmental benefits │ and improve return on investments;   deliver │                  │
│ and reduce energy consumption by alleviating  │ environmental benefits and reduce energy     │                  │
│ congestion and streamlining traffic flow;     │ consumption by alleviating congestion and    │                  │
│ measure and improve the operational           │ streamlining traffic flow;   measure and     │                  │
│ performance of its transportation network;    │ improv

***** eval metrics *****
  epoch                   =     1.9997
  eval_accuracy           =     0.4286
  eval_loss               =     0.6916
  eval_runtime            = 0:00:28.84
  eval_samples_per_second =     19.585
  eval_steps_per_second   =      2.461


/home/gurovev/dev/rlhf-and-friends/FedPPO/.fed-ppo-venv/lib/python3.10/site-packages/trl/trainer/utils.py:785: UserWarning: There are 453 out of 565 instances where the predictions for both options are equal. These instances are ignored in the accuracy computation.
  warnings.warn(
